In [96]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog


# Open a file selection dialog and get the selected file's path
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

# Load the dataframe
df = pd.read_excel(file_path, sheet_name="GISLeaseUpdateResults")
df.head(5)

,Internal ID,Lease Type,Lease Subtype,Transaction Number,Lessee(s),Legacy Lease Number,Start Date (Letter Merge),End Date (Letter Merge),Related Asset,Name,Acreage,Legal Description,Lease Terms (Years),Administrator,District,Meridian,Township,Range,Section#,Lease Status
0,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,70.00,E2,15.0,Rachel Turner,North Central,6,5S,59W,10,Active
1,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002094,6-5S-59W-36-Surface-Arapahoe-School,640.00,ALL,15.0,Rachel Turner,North Central,6,5S,59W,36,Active
2,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,250.00,E2,15.0,Rachel Turner,North Central,6,5S,59W,10,Active
3,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004437,6-15S-62W-1-Surface-El Paso-School,637.12,"LTS 1-2, S2NE, SE, W2",25.0,Justin Osborne,South Central,6,15S,62W,1,Active
4,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004635,6-16S-62W-1-Surface-El Paso-School,609.18,"LTS 1-4, S2N2, S2",25.0,Justin Osborne,South Central,6,16S,62W,1,Active


In [97]:
# Change Column Names
column_name_changes = {'Internal ID': 'Netsuite Lease ID',
                      'Start Date (Letter Merge)': 'Start Date',
                      'End Date (Letter Merge)': 'End Date',
                      'Related Asset': 'Related FAM(s)',
                      'Name': 'Related FAM Name',
                      'Legal Description': 'Lease Legal Description',
                      'Section#': 'Section'}

# Use the rename method to change the column names
df = df.rename(columns=column_name_changes)

df.head(5)

,Netsuite Lease ID,Lease Type,Lease Subtype,Transaction Number,Lessee(s),Legacy Lease Number,Start Date,End Date,Related FAM(s),Related FAM Name,Acreage,Lease Legal Description,Lease Terms (Years),Administrator,District,Meridian,Township,Range,Section,Lease Status
0,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,70.00,E2,15.0,Rachel Turner,North Central,6,5S,59W,10,Active
1,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002094,6-5S-59W-36-Surface-Arapahoe-School,640.00,ALL,15.0,Rachel Turner,North Central,6,5S,59W,36,Active
2,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,250.00,E2,15.0,Rachel Turner,North Central,6,5S,59W,10,Active
3,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004437,6-15S-62W-1-Surface-El Paso-School,637.12,"LTS 1-2, S2NE, SE, W2",25.0,Justin Osborne,South Central,6,15S,62W,1,Active
4,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004635,6-16S-62W-1-Surface-El Paso-School,609.18,"LTS 1-4, S2N2, S2",25.0,Justin Osborne,South Central,6,16S,62W,1,Active


In [98]:
# Group the dataframe by the Lease Number column and sum the values in the Acreage column
summary_df = df.groupby('Transaction Number')['Acreage'].sum().reset_index()

# Rename the Acreage column to Total Lease Acreage
summary_df = summary_df.rename(columns={'Acreage': 'Total Lease Acreage'})

# The summary dataframe now contains the total acreage for each Lease Number
summary_df

,Transaction Number,Total Lease Acreage
0,AG-100001,960.000
1,AG-100014,47859.790
2,AG-100112,2660.550
3,AG-100145,1280.000
4,AG-100148,5175.020
...,...,...
4176,TOW-112838,40.000
4177,TOW-113440,0.028
4178,TOW-113960,160.000
4179,TOW-114153,0.370


In [99]:
df = pd.merge(df, summary_df, on='Transaction Number')

df.head(5)

,Netsuite Lease ID,Lease Type,Lease Subtype,Transaction Number,Lessee(s),Legacy Lease Number,Start Date,End Date,Related FAM(s),Related FAM Name,...,Lease Legal Description,Lease Terms (Years),Administrator,District,Meridian,Township,Range,Section,Lease Status,Total Lease Acreage
0,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,...,E2,15.0,Rachel Turner,North Central,6,5S,59W,10,Active,960.00
1,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002094,6-5S-59W-36-Surface-Arapahoe-School,...,ALL,15.0,Rachel Turner,North Central,6,5S,59W,36,Active,960.00
2,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,...,E2,15.0,Rachel Turner,North Central,6,5S,59W,10,Active,960.00
3,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004437,6-15S-62W-1-Surface-El Paso-School,...,"LTS 1-2, S2NE, SE, W2",25.0,Justin Osborne,South Central,6,15S,62W,1,Active,47859.79
4,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004635,6-16S-62W-1-Surface-El Paso-School,...,"LTS 1-4, S2N2, S2",25.0,Justin Osborne,South Central,6,16S,62W,1,Active,47859.79


In [103]:
# Groupby Lease Legal Descriptions by Transaction number. 

df_legals = df.groupby('Transaction Number')['Lease Legal Description'].apply(lambda x: ', '.join([str(i) for i in x])).reset_index()
df_legals.columns = ['Transaction Number', 'Lease Legal Description']

# Nan values need to be dropped
df_legals['Lease Legal Description'] = df_legals['Lease Legal Description'].str.replace('nan, ', '')
df_legals['Lease Legal Description'] = df_legals['Lease Legal Description'].str.replace('nan', '')
df_legals['Lease Legal Description'] = df_legals['Lease Legal Description'].str.replace(', nan', '')

# Replace values with "See Lease Documentation" if greater than 50
mask = df_legals['Lease Legal Description'].str.len() > 50
df_legals['Lease Legal Description'] = df_legals['Lease Legal Description'].where(~mask, 'See Lease Documentation')

df_legals


,Transaction Number,Lease Legal Description
0,AG-100001,"E2, ALL, E2"
1,AG-100014,See Lease Documentation
2,AG-100112,See Lease Documentation
3,AG-100145,"ALL, ALL, ALL, ALL"
4,AG-100148,See Lease Documentation
...,...,...
4176,TOW-112838,ALL
4177,TOW-113440,SESWSE
4178,TOW-113960,NW
4179,TOW-114153,NENESE


In [181]:
# Merge df and df_legals into merged_df
merged_df = pd.merge(df, df_legals, on='Transaction Number', how='outer')
merged_df = merged_df.rename(columns={'Lease Legal Description_y': 'Lease Legal Description(s)'})

merged_df.head(5)

,Netsuite Lease ID,Lease Type,Lease Subtype,Transaction Number,Lessee(s),Legacy Lease Number,Start Date,End Date,Related FAM(s),Related FAM Name,...,Lease Terms (Years),Administrator,District,Meridian,Township,Range,Section,Lease Status,Total Lease Acreage,Lease Legal Description(s)
0,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,...,15.0,Rachel Turner,North Central,6,5S,59W,10,Active,960.00,See Lease Documentation
1,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002094,6-5S-59W-36-Surface-Arapahoe-School,...,15.0,Rachel Turner,North Central,6,5S,59W,36,Active,960.00,See Lease Documentation
2,8601,Agriculture,Agriculture,AG-100001,"CLARENCE G ELDRINGHOFF, SUSAN ELDRINGHOFF",AG 39761,2008-05-12,2023-05-12,FAM002093,6-5S-59W-10-Surface-Arapahoe-School,...,15.0,Rachel Turner,North Central,6,5S,59W,10,Active,960.00,See Lease Documentation
3,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004437,6-15S-62W-1-Surface-El Paso-School,...,25.0,Justin Osborne,South Central,6,15S,62W,1,Active,47859.79,See Lease Documentation
4,8704,Agriculture,Grazing,AG-100014,"NATURE CONSERVANCY, DICK TANNER, SANDRA J TANN...",AG 42264,2006-11-27,2023-02-27,FAM004635,6-16S-62W-1-Surface-El Paso-School,...,25.0,Justin Osborne,South Central,6,16S,62W,1,Active,47859.79,See Lease Documentation


In [177]:
grouped_df = df.groupby('Transaction Number')['Section'].apply(lambda x: ', '.join(x) if x.nunique()>1 else x.iloc[0]).reset_index()



grouped_df['Section'] = grouped_df['Section'].str.replace('nan, ', '')
grouped_df['Section'] = grouped_df['Section'].str.replace('nan', '')
grouped_df['Section'] = grouped_df['Section'].str.replace(', nan', '')


# Replace values with "See Lease Documentation" if greater than 20
mask = grouped_df['Section'].str.len() > 2
grouped_df['Section'] = grouped_df['Section'].where(~mask, 'See Lease Documentation')


grouped_df.sample(20)

,Transaction Number,Section
2573,OG-108405,See Lease Documentation
2985,OG-113123,See Lease Documentation
217,AG-101883,See Lease Documentation
1611,AG-112821,16
1434,AG-111926,See Lease Documentation
3752,SM-102424,36
2242,EP-110714,36
1658,AG-113066,32
74,AG-101713,36
3135,OG-114136,See Lease Documentation


In [183]:
# I need to merge or join the results of grouped_df into merged_df

merged_df.dtypes

Netsuite Lease ID                      int64
Lease Type                            object
Lease Subtype                         object
Transaction Number                    object
Lessee(s)                             object
Legacy Lease Number                   object
Start Date                    datetime64[ns]
End Date                      datetime64[ns]
Related FAM(s)                        object
Related FAM Name                      object
Acreage                              float64
Lease Legal Description_x             object
Lease Terms (Years)                  float64
Administrator                         object
District                              object
Meridian                              object
Township                              object
Range                                 object
Section                               object
Lease Status                          object
Total Lease Acreage                  float64
Lease Legal Description(s)            object
dtype: obj